## **N Sree Dhyuti - Submission**

# Step 1: Load Data

In [1]:
# Loading Data
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

data_folder = r"C:\Users\dhyut\SEM8\SequrAise Intern Assignment\data"

def load_images_from_folder():
    cap = cv2.VideoCapture(r"C:\Users\dhyut\SEM8\SequrAise Intern Assignment\AI Assignment video.mp4")
    i = 0
    frames = [] 
    #3182
    while(cap.isOpened() and i < 3182):
        ret, frame = cap.read()
        if ret == False:
            break
        frames.append(frame)
        i += 1
    cap.release()
    cv2.destroyAllWindows()    
    return frames

images_data = load_images_from_folder()

**Function to get segment each quadrant**

In [2]:
def get_quadrants(img):
#     ROI = img_sample[0:1020, 770:1750]
    q1 = img[570:970, 1260:1725]
    q2 = img[540:1000, 790:1210]
    q3 = img[20:500, 795:1220]
    q4 = img[20:500, 1260:1735]
    return [q1, q2, q3, q4]


**Setting up hsv upper and lower limits of each of the following colors**

In [3]:
# hsv lower and upper limits of colors involved in dataset

# Manually encoded hsv limits
red = [np.array([0, 100, 40]), np.array([60, 255, 255])]

blue = [np.array([94, 80, 2]), np.array([126, 255, 255])]

green = [np.array([25, 52, 72]), np.array([102, 255, 255])]

black = [np.array([0, 0, 0]), np.array([360, 255, 67])]

**Function to find ball color in the given quadrant**

In [4]:
def ball_color(img):
    possible_colors = [red, blue, green, black]
    counts = []
    color = ""
    for i in range(len(possible_colors)):
        image1 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(image1, possible_colors[i][0], possible_colors[i][1])
        counts.append(np.count_nonzero((mask == 255)))
    # Thresholding 
    if(counts[2] > 8000):
        color = "Green"
    elif(counts[0] > 7500):
        color = "Orange"
    elif((counts[2] > 4000) and (counts[1] > 750)):
        color = "Yellow"
    elif((counts[1] > 900) and (counts[2] < 100) and counts[3] < 50):
        color = "None"
    else:
        color = "White"
    return color

**Running algorithm on all frames in video (Time Taking)**

In [5]:
final_imgs = []
text_content = []
cnt = 0
for i in images_data:
    cnt += 1
    string = ""
    image = i
    imag = get_quadrants(image)
    q1 = ball_color(imag[0])
    q2 = ball_color(imag[1])
    q3 = ball_color(imag[2])
    q4 = ball_color(imag[3])
    if(cnt == 1):
        prev_q1 = q1
        prev_q2 = q2
        prev_q3 = q3
        prev_q4 = q4
        
    def entry(prev_q, q):
        if(prev_q == q):
            return "entry"
        else:
            return "exit"
        val = "entry"
    string_for_txt_file = str(cnt) + ", " + "1, " + q1 + ", " + entry(prev_q1, q1)
    text_content.append(string_for_txt_file)
    string_for_txt_file = str(cnt) + ", " + "2, " + q2 + ", " + entry(prev_q2, q2)
    text_content.append(string_for_txt_file)
    string_for_txt_file = str(cnt) + ", " + "3, " + q3 + ", " + entry(prev_q3, q3)
    text_content.append(string_for_txt_file)
    string_for_txt_file = str(cnt) + ", " + "4, " + q4 + ", " + entry(prev_q4, q4)
    text_content.append(string_for_txt_file)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    string = "Quadrant 1: " + q1 
    image = cv2.putText(image, string , org, font, fontScale, color, thickness, cv2.LINE_AA)
    string = "Quadrant 2: " + q2 
    org = (50, 80)
    image = cv2.putText(image, string , org, font, fontScale, color, thickness, cv2.LINE_AA)
    string = "Quadrant 3: " + q3 
    org = (50, 110)
    image = cv2.putText(image, string , org, font, fontScale, color, thickness, cv2.LINE_AA)
    string = "Quadrant 4: " + q4 
    org = (50, 140)
    image = cv2.putText(image, string , org, font, fontScale, color, thickness, cv2.LINE_AA)
    prev_q1 = q1
    prev_q2 = q2
    prev_q3 = q3
    prev_q4 = q4
    
#     text_content.append(string_for_txt_file)
    final_imgs.append(image)

**Compiling all the frames to a video**

In [6]:
import cv2
import os

# image_folder = 'images'
video_name = 'video1.avi'

# frame = cv2.imread(os.path.join(image_folder, final_imgs[0]))
height, width, layers = final_imgs[0].shape

video = cv2.VideoWriter(video_name, 0, 30.30, (width,height))

for image in final_imgs:
    video.write(image)

cv2.destroyAllWindows()
video.release()

**Creating a text file for the same**

In [8]:
string = "frame, quadrant, color, entry/exit \n"

for i in text_content:
    string += i + "\n"

file_object = open('results.txt', 'a')
file_object.write(string)
file_object.close()